In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from jax import random, numpy as jnp
from stadion.models import LinearSDE, MLPSDE
from pprint import pprint
from stadion import kds_loss
import numpy as onp
import ot

2025-03-13 15:23:04.149953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741893784.160629 2821983 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741893784.163889 2821983 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from nn import NNSDE
from nonlinear_script import *

In [3]:
n = 2000
d = 20
r = 3
n_envs = 10
n_test_envs = 10
epsilon = 0.2
gamma = 0.98
activation = "mixed"
model_activation = "learned"

model_hidden_size = 4

bandwidth = 7.0
steps = 50000
estimator = "linear"
reg = 0.000

scale = 1e-1
intv_scale = 0.3

n_samples_burnin=200
learning_rate = 0.003
weight_decay = 0.0001

seed = 12

In [4]:
key = random.PRNGKey(seed)

true_model = build_model(r, activation, epsilon, gamma, n_samples_burnin)
key, true_model, param = initialize_model(key, true_model, d)

# param["mlp_0"] = jax.numpy.eye(d, r)
# param["mlp_1"] = jax.numpy.eye(r, d)
# true_model.param = param

key, intv_param, datasets, targets = build_data(key, true_model, d, n_envs, intv_scale, n)
key, test_intv_param, test_datasets, test_targets = build_data(key, true_model, d, n_test_envs, intv_scale, n)


model = build_model(model_hidden_size, model_activation, epsilon, gamma, n_samples_burnin)
key, model = fit_model(key, model, datasets, targets, intv_param, bandwidth, steps, estimator,
                       reg, learning_rate, weight_decay, scale)

W0000 00:00:1741893816.504022 2821983 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


step:  5000 kds:     0.000498  | min remain:  2.1  sec/step: 0.003
step: 10000 kds:     0.000039  | min remain:  1.6  sec/step: 0.002
step: 15000 kds:     0.000038  | min remain:  1.4  sec/step: 0.002
step: 20000 kds:     0.000035  | min remain:  1.2  sec/step: 0.002
step: 25000 kds:     0.000039  | min remain:  1.0  sec/step: 0.002
step: 30000 kds:     0.000029  | min remain:  0.8  sec/step: 0.002
step: 35000 kds:     0.000025  | min remain:  0.6  sec/step: 0.002
step: 40000 kds:     0.000029  | min remain:  0.4  sec/step: 0.002
step: 45000 kds:     0.000021  | min remain:  0.2  sec/step: 0.002
step: 50000 kds:     0.000026  | min remain:  0.0  sec/step: 0.002


In [5]:
wds, mean, std = assess_model_mean(key, model, n_test_envs, test_intv_param, test_datasets, n)
print(wds)
print(mean, std)

[ 6.638041   5.8263144  7.731132   8.976746   6.3155036  6.7212167
  8.138009   1.4022197 10.070792   8.458597 ]
7.0278573 2.2511466


In [6]:
wds, mean, std = assess_model_mean(key, true_model, n_test_envs, test_intv_param, test_datasets, n)
print(wds)
print(mean, std)

[0.80941457 0.56436753 0.49097773 0.4865995  0.87020886 0.44422337
 0.4845277  0.35852554 0.6442967  0.9672584 ]
0.61204 0.19343834


In [7]:
# #W2
# wds, mean, std = assess_model(key, model, n_test_envs, test_intv_param, test_datasets, n, ot_epsilon = 0.2)
# print(wds)
# print(mean, std)

# wds, mean, std = assess_model(key, true_model, n_test_envs, test_intv_param, test_datasets, n, ot_epsilon = 0.2)
# print(wds)
# print(mean, std)

In [8]:
#s:13.926264 5.4777803

#s2:
#l:
#l2:
#t:0.61204 0.19343834

#t2:

In [9]:
#s:
#s2:
#l:
#l2:
#t:
#t2:

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

if d == 2:

    total = 4

    pred_datasets = []
    for k in range(total):
        local_intv_param = intv_param.index_at(k)
        key, subk = random.split(key)
        data = model.sample(subk, n_samples = n, intv_param = local_intv_param)
        pred_datasets.append(data)

    fig, axes = plt.subplots(nrows=total,ncols=2,sharex=True, sharey=True)

    
    for k in range(total):
    
        x = datasets[k][:,0]
        y = datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,0], fill=True,cmap="Purples_r")
        axes[k,0].set_title(f"True model on training env {k}")
        
        x = pred_datasets[k][:,0]
        y = pred_datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,1], fill=True,cmap="Purples_r")
        axes[k,1].set_title(f"Trained model on training env {k}")

    plt.tight_layout()
    plt.show()


In [11]:
if d == 2:

    total = 4

    pred_test_datasets = []
    for k in range(total):
        local_intv_param = test_intv_param.index_at(k)
        key, subk = random.split(key)
        data = model.sample(subk, n_samples = n, intv_param = local_intv_param)
        pred_test_datasets.append(data)

    fig, axes = plt.subplots(nrows=total,ncols=2,sharex=True, sharey=True)

    for k in range(total):
    
        x = test_datasets[k][:,0]
        y = test_datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,0], fill=True,cmap="Purples_r")
        axes[k,0].set_title(f"True model on test env {k}")
        
        x = pred_test_datasets[k][:,0]
        y = pred_test_datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,1], fill=True,cmap="Purples_r")
        axes[k,1].set_title(f"Trained model on test env {k}")

    plt.tight_layout()
    plt.show()

In [12]:
print(model.param)

ModelParameters
{'A': Array([[ 1.06135150e-03,  1.84137318e-02,  7.91915059e-02,
        -4.85216454e-02, -2.41584942e-01,  2.25841310e-02,
         7.05641881e-02, -2.40996391e-01,  2.27144107e-01,
         1.76713124e-01, -1.64344981e-01,  4.12784107e-02,
         2.17562601e-01,  2.03528658e-01,  1.72908455e-02,
         1.15032642e-04,  1.15673222e-01, -1.04073860e-01,
        -2.57471919e-01,  1.03393197e-03],
       [-7.94505626e-02, -2.08376974e-01, -2.28921622e-01,
        -1.13350756e-01,  3.18713747e-02, -1.35422740e-02,
        -1.55948058e-01,  8.10237899e-02, -8.12917724e-02,
        -2.35335380e-02, -2.02927202e-01, -2.48189624e-02,
        -9.38093066e-02, -8.89905766e-02, -1.92376137e-01,
        -1.84606418e-01, -5.79361804e-02, -7.22021610e-02,
        -1.84247419e-01, -2.30497181e-01],
       [-5.40005565e-02,  2.52133846e-01, -2.37013353e-03,
        -1.69623077e-01, -2.32304394e-01, -1.20850094e-01,
        -6.49565384e-02,  1.33082226e-01,  1.83534637e-01,
       

In [13]:
print(true_model.param)

ModelParameters
{'log_reversion': Array([ 0., -0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,
        0., -0.,  0., -0., -0.,  0., -0.], dtype=float32),
 'mlp_0': Array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32),
 'mlp_1': Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32),
 'mlp_b_0': Array([ 0.,  0., -0.], dtype=float32),
 'mlp_b_1': Array([